In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
import matplotlib.pyplot as pl
from bson.json_util import dumps
import base64

# importing our CRUD class from your animalShelter file
from animalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
#Setting the username and password for your mongoDB
username = "aacuser"
password = "bikas123"
#instantiate the AnimalShelter object
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object
df = pd.DataFrame.from_records(shelter.read_all({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#adding grazioso salvare's logo 
image_filename = 'logo.png'  
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# fixing the html image tag
app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'height': '196px', 'width': '196px'}
        )
    ),

    # adding unique identifier with name and date
    html.Center(html.H1('Aakash Thapa 04/14/2023')),
    html.Hr(),

    html.Div([
        # adding code for filtering options
        dcc.Dropdown(
            id='demo-dropdown',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='water'
        ),
        html.Div(id='dd-output-container')
    ]),

    html.Hr(),

    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i} for i in df.columns
        ],
        data=df.to_dict('records'),
        # code from ModuleSix which set up the features for interactive data-table
        editable=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=6,
    ),

    html.Br(),
    html.Hr(),

  #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 dcc.Graph(
                     id="pie-chart",
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])


#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(
    Output('datatable-id', 'data'),
    [dash.dependencies.Input('demo-dropdown', 'value')])
def update_output(value):
    if (value =='water'):
        df = pd.DataFrame(list(shelter.read_all(
            { 
            "breed": {
                "$in": [
                    "Chesapeake Bay Retriever",
                    "Labrador Retriever Mix",
                    "Newfoundland"
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
    elif (value == 'mountain'):
        df = pd.DataFrame(list(shelter.read_all(
            { 
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Husky",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 156.0
                    }
                }
            ]
        }
        )))
    elif (value == 'disaster'):
        df = pd.DataFrame(list(shelter.read_all(
            { 
            "breed": {
                "$in": [
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 20.0
            },
            "$and": [
                {
                    "age_upon_outcome_in_weeks": {
                        "$lte": 300.0
                    }
                }
            ]
        }
        )))
    else:
        df = pd.DataFrame.from_records(shelter.read_all({}))
    return df.to_dict('records')
    


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

## Pie-Chart

@app.callback(
    Output("pie-chart", "figure"),
    [Input("datatable-id", "data")]
)
def generate_chart(data):
    #code for pie-chart
    dff = pd.DataFrame.from_dict(data)
    fig = px.pie(dff,names='breed')
    return fig


#### Map ####

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, rows):
    viewDF = pd.DataFrame.from_dict(viewData)
    dff = viewDF.loc[rows]
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                #dl.Tooltip(dff.iloc[0, 4]),
                #dl.Tooltip(viewData.iloc[0, 4]),
                dl.Tooltip(dff['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    #html.P(dff.iloc[1, 9])
                    html.P(dff.loc[0,'name'])
                ])
            ])
        ])
    ]


app

username and password are: aacuser bikas123
Connection was successful
